<a href="https://colab.research.google.com/github/proap900/AI_LABEXAM_Assignment/blob/main/block_arrangement_by_hill_climbing_and_heuristic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, clear_output
import ipywidgets as widgets

# ✅ Define output widget FIRST
output = widgets.Output()

# --- Configuration ---
GOAL = ['A', 'B', 'C', 'D']
COLORS = {'A': 'red', 'B': 'green', 'C': 'blue', 'D': 'orange'}

# --- Heuristic & Neighbors ---
def heuristic(state):
    return sum(1 for i in range(4) if state[i] != GOAL[i])

def neighbors(state):
    result = []
    for i in range(len(state) - 1):
        new = state[:]
        new[i], new[i+1] = new[i+1], new[i]
        result.append((new, f"Swap {i}-{i+1}"))
    return result

# --- Visualization ---
def draw_stack(state, move=None, h_val=None):
    output.clear_output(wait=True)
    with output:
        fig, ax = plt.subplots(figsize=(2, 5))
        ax.set_xlim(0, 2)
        ax.set_ylim(0, 4)
        ax.axis('off')

        for i in range(len(state)):
            block = state[i]
            rect = patches.Rectangle((0.5, 3 - i), 1, 1, linewidth=1, edgecolor='black', facecolor=COLORS[block])
            ax.add_patch(rect)
            ax.text(1.0, 3 - i + 0.5, block, fontsize=16, ha='center', va='center', color='white')

        title = f"{move} | h: {h_val}" if move else f"Start | h: {h_val}"
        plt.title(title)
        plt.show()

# --- Stepper Logic ---
class BlockStepper:
    def __init__(self):
        self.init_ui()

    def init_ui(self):
        self.state = self.random_stack()
        self.h = heuristic(self.state)
        self.done = False
        draw_stack(self.state, "Start", self.h)

    def random_stack(self):
        st = GOAL[:]
        while True:
            random.shuffle(st)
            if st != GOAL:
                return st

    def next_step(self, b):
        if self.done: return
        for neighbor, move in neighbors(self.state):
            h_new = heuristic(neighbor)
            if h_new < self.h:
                self.state = neighbor
                self.h = h_new
                draw_stack(self.state, move, self.h)
                if self.h == 0:
                    with output:
                        print("🎉 Goal Reached!")
                    self.done = True
                return
        with output:
            print("⚠️ Stuck in local maximum!")
            print("🔁 Click 'Shuffle Again' to restart.")
        self.done = True

    def restart(self, b):
        self.init_ui()
        self.done = False

# --- Widgets ---
stepper = BlockStepper()
next_button = widgets.Button(description="Next Swap 🔄")
reset_button = widgets.Button(description="Shuffle Again 🎲")

next_button.on_click(stepper.next_step)
reset_button.on_click(stepper.restart)

ui = widgets.VBox([next_button, reset_button, output])
display(ui)